##BASIC CHATBOT (Alter based on requirments)

In [ ]:
#types of flan-t5 model
'''
flan-t5-small
flan-t5-base
flan-t5-large
flan-t5-xl
flan-t5-xxl
'''

In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

model_name = "google/flan-t5-large"
# model_name = "google/flan-t5-base"
# model_name="google/flan-t5-xxl"

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name,use_fast=True)

conversation_history=[]

def generate_text(prompt, conversation_history, max_length=400, num_return_sequences=1):
    if not model or not tokenizer:
        raise ValueError("Model and tokenizer are not loaded. Please load them before using this function.")

    conversation_history.append(prompt)
    full_conversation='\n'.join(conversation_history)

    inputs = tokenizer(full_conversation, return_tensors="pt").to(device)

    with torch.no_grad():
      outputs = model.generate(
          **inputs,
          max_length=max_length,
          num_beams=2,#6
          no_repeat_ngram_size=1,
          num_return_sequences=num_return_sequences,
          length_penalty=1.8,
          early_stopping=True
      )

    generated_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return generated_texts

chatbot_role = "Technician Support Chatbot"
chatbot_instructions = """
You are a technician support chatbot.
Your primary goal is to assist users by identifying their technical problems and suggesting suitable technicians.
Additionally, you should handle other queries related to the application.

Your responses should be:
* Clear and concise: Avoid unnecessary jargon or overly complex explanations.
* Helpful and informative: Provide relevant and useful information to the user.
* Friendly and approachable: Maintain a positive and welcoming tone.
* Accurate and reliable: Ensure your responses are correct and trustworthy.
* Contextual: Consider the context of the conversation and tailor your responses accordingly.
* Domain-specific: Keep your responses within the application domain.

Your domain includes the following technician services:
- Electrician
- Plumber
- Carpenter
- Cleaner
- Painter
- AC & HVAC Technician
- Sewage and Drain Cleaner
- Welding and Metalwork
- Emergency Generator and Invertor Maintenance Worker

For example:
- **User:** My tap is not working.
- **Chatbot:** It sounds like you're experiencing a plumbing issue. For a problem with your tap, I would recommend contacting a plumber. Plumbers are skilled in repairing and maintaining various plumbing fixtures and can help ensure everything is functioning correctly. Would you like me to assist you in finding a plumber in your area?

- **User:** My tube light is not working.
- **Chatbot:** It sounds like you're having an issue with your tube light. For problems like this, I would suggest getting in touch with an electrician. Electricians are trained to handle electrical issues and can help diagnose and fix the problem efficiently. Would you like me to assist you in finding an electrician nearby?

- **User:** My inverter is not working.
- **Chatbot:** It sounds like you're having an issue with your inverter. For problems like this, I would recommend contacting an Emergency Generator and Invertor Maintenance Worker. These professionals are trained to handle issues with inverters and can help diagnose and fix the problem efficiently. Would you like me to assist you in finding a technician nearby?

Remember to stay focused on providing application-related support and technician recommendations.
"""

user_input = "light is not glowing "
prompt = f"{chatbot_instructions}\n\n**Role:** {chatbot_role}\n\n**User:** {user_input}\n\n**Chatbot:**"
generated_texts = generate_text(prompt,conversation_history)
print("Generated responses:")

for text in generated_texts:
    print(f"Bot: {text}")

Generated responses:
Bot: I would suggest getting in touch with an electrician.


In [2]:
user_input = "i need to paint my house "
prompt = f"{chatbot_instructions}\n\n**Role:** {chatbot_role}\n\n**User:** {user_input}\n\n**Chatbot:**"
generated_texts = generate_text(prompt,conversation_history)
print("Generated responses:")

for text in generated_texts:
    print(f"Bot: {text}")

Token indices sequence length is longer than the specified maximum sequence length for this model (1020 > 512). Running this sequence through the model will result in indexing errors


Generated responses:
Bot: I can help you with that. What kind of paint do u want to use?


In [ ]:
user_input = "my washing machine is not working"
prompt = f"{chatbot_instructions}\n\n**Role:** {chatbot_role}\n\n**User:** {user_input}\n\n**Chatbot:**"
generated_texts = generate_text(prompt,conversation_history)
print("Generated responses:")

for text in generated_texts:
    print(f"Bot: {text}")

Generated responses:
Bot: It sounds like you're having an issue with your washing machine. For problems such as this, I would suggest contacting one of the following technicians:


In [ ]:
user_input = "my invertor is not working "
prompt = f"{chatbot_instructions}\n\n**Role:** {chatbot_role}\n\n**User:** {user_input}\n\n**Chatbot:**"
generated_texts = generate_text(prompt,conversation_history)
print("Generated responses:")

for text in generated_texts:
    print(f"Bot: {text}")

Generated responses:
Bot: It sounds like you're having an issue with your inverter. For problems such as this, I would recommend contacting Emergency Generator and Inveror Maintenance Workers


In [3]:
while True:
    user_input = input("Enter your query (type 'quit' to exit): ")
    if user_input.lower() == "quit":
        print("Exiting the chatbot. Have a great day!")
        break

    prompt = f"{chatbot_instructions}\n\n**Role:** {chatbot_role}\n\n**User:** {user_input}\n\n**Chatbot:**"
    generated_texts = generate_text(prompt,conversation_history)
    print("Generated responses:")

    for text in generated_texts:
        print(f"Bot: {text}")

In [ ]:


############################################################################### sentiment analysis ###############################################################


##MODEL 1


In [36]:
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')


sia=SentimentIntensityAnalyzer()
def analyze_sentiment(text):
  score=sia.polarity_scores(str(text))
  if score['compound']>=0.05:
    return 'Positive'
  elif score['compound']<=-0.05:
    return 'Negative'
  else:
    return 'Neutral'

inputs='The technician work well'
text_senti=analyze_sentiment(inputs)
print(text_senti)

aspect1=int(input('Professionalism - Was the technician polite and respectful? (1 to 5): '))
aspect2=int(input('Expertise – Did the technician demonstrate knowledge and skill? (1 to 5): '))
aspect3=int(input('Timeliness – Was the service completed on time? (1 to 5): '))
aspect4=int(input('Communication – Did the technician clearly explain the issue and solution? (1 to 5): '))
aspect5=int(input('Work Quality – Was the issue resolved effectively and satisfactorily? (1 to 5): '))
aspect_score=(aspect1+aspect2+aspect3+aspect4+aspect5)/5
print(aspect_score)

if text_senti=='Positive' and aspect_score>=4:
  print('Positive')
elif text_senti=='Negative' and aspect_score<=2:
  print('Negative')
else:
  print('Neutral')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Positive
Professionalism - Was the technician polite and respectful? (1 to 5): 3
Expertise – Did the technician demonstrate knowledge and skill? (1 to 5): 3
Timeliness – Was the service completed on time? (1 to 5): 3
Communication – Did the technician clearly explain the issue and solution? (1 to 5): 3
Work Quality – Was the issue resolved effectively and satisfactorily? (1 to 5): 3
3.0
Neutral


##MODEL 2


In [37]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")
print(sentiment_pipeline("The technician work well"))


Device set to use cpu


[{'label': '4 stars', 'score': 0.5136815905570984}]


##MODEL 3


In [35]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

inputs = tokenizer("The technician work well", return_tensors="pt")
outputs = model(**inputs)
output = outputs.logits

# Get the predicted class index
predicted_class_index = torch.argmax(output, dim=1).item()

# Map the class index to the sentiment label
sentiment_labels = ['negative', 'neutral', 'positive']
predicted_sentiment = sentiment_labels[predicted_class_index]

print(f"Predicted Sentiment: {predicted_sentiment}")

Predicted Sentiment: positive


##MODEL 4

In [39]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
print(sentiment_pipeline("The technician work well"))


Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.99980229139328}]


##MODEL 5

In [17]:
from textblob import TextBlob
print(TextBlob("The technician didnt work well").sentiment)


Sentiment(polarity=0.0, subjectivity=0.0)


##MODEL 6

In [43]:
from transformers import pipeline
import socket

# Increase the timeout value
socket.setdefaulttimeout(30)  # Set timeout to 30 seconds

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
result = classifier(
    "The technician didn't work well",
    candidate_labels=["positive", "negative", "neutral"],
)

# Print only the highest scoring label
print(result["labels"][0])


Device set to use cpu


negative


##MODEL 7

In [ ]:
!pip install -q flair

In [3]:
from flair.nn import Classifier
from flair.data import Sentence

sentence = Sentence("The technician didn't work well")
classifier = Classifier.load('sentiment')
classifier.predict(sentence)

predicted_label = sentence.labels[0].value
print(predicted_label)


NEGATIVE


##MODEL 8

In [4]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")
print(sentiment_pipeline("The technician work well"))


[{'label': '4 stars', 'score': 0.5136815905570984}]
